In [1]:
import os

# define the directories where the imgaes are

pizza_directory="./pizza/"
not_pizza_directory="./not_pizza/"

# get the images from the directories

not_pizza_files=[[0, not_pizza_directory, i] for i in os.listdir(not_pizza_directory)]
pizza_files=[[1, pizza_directory, i] for i in os.listdir(pizza_directory)]

print("Number of pizza images:",len(pizza_files))
print("Number of non-pizza images:",len(not_pizza_files))

# combine the image sets

all_files = not_pizza_files + pizza_files

print("Total number of images:", len(all_files))

# each element in all_files is of the format [is_pizza: boolean, directory: string, filename: string]
# in other words, for each `img` in `all_files`:
#    - img[0] = 0 if not pizza, 1 if pizza
#    - img[1] = directory where the image is
#    - img[2] = file name

Number of pizza images: 983
Number of non-pizza images: 983
Total number of images: 1966


#

In [8]:
from PIL import Image

import numpy as np

standardSize = (512,512)

X = resizedImages = list()
y = list()

# structure is:
# resizedImages[i] = 2-D numpy array of pixels, [512][512][3]


PATH = os.path.abspath(os.getcwd())

dirExists = os.path.exists(f'{PATH}/resizedImages')

if (not dirExists):
    os.mkdir('resizedImages')

for i, image in enumerate(all_files):
    imgData = Image.open(image[1] + image[2])
    imgData = imgData.resize(standardSize)
    imgData.save(f'{PATH}/resizedImages/{i}.jpg')
    # storing images
    resizedImages.append(np.array(imgData))
    y.append(image[0])

# ! Takes about 30 seconds...

In [9]:
import pandas as pd

X = np.array(resizedImages)
y = np.array(y)
print(len(X))
   
print(len(y))

print(X.shape)
print(y.shape)

1966
1966
(1966, 512, 512, 3)
(1966,)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

print(X_train.shape)
print(X_test.shape)

(1572, 512, 512, 3)
(394, 512, 512, 3)


In [62]:
from keras.layers import Conv2D, Flatten, Dense
from keras.models import Sequential
model = Sequential() # Initialising the ANN
# build your convulusional layers here:

imgShape = (512, 512, 3)

model.add(Conv2D(4, 3, input_shape=imgShape, activation = 'relu'))
model.add(Conv2D(3, 3, input_shape=imgShape, activation = 'relu'))
model.add(Conv2D(2, 3, input_shape=imgShape, activation = 'relu'))


# make sure you flatten after you convulusional layers here:
model.add(Flatten())

# Feel free to add hidden layers here: 

# model.add(Dense(units = 10, activation = 'relu'))

# # Hint reduce to a 1d vector as you have in the previous exercises.
model.add(Dense(units = 1, activation = 'sigmoid'))

# compile your model here:

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

numEpoches = 2
printOutput = 5

modelData = model.fit(X_train, y_train, batch_size = 1572, epochs = numEpoches, verbose = 1)

Epoch 1/2


In [61]:
from sklearn.metrics import classification_report

yhat = model.predict(X_test)
yhat = [0 if y>=0.5 else 1 for y in yhat]

print(classification_report(y_test, yhat))

13/13 [==============================] - 6s 427ms/step
              precision    recall  f1-score   support

           0       0.45      0.51      0.48       197
           1       0.43      0.37      0.40       197

    accuracy                           0.44       394
   macro avg       0.44      0.44      0.44       394
weighted avg       0.44      0.44      0.44       394

